In [1]:
spark

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 50, 6, Finished, Available, Finished)

In [2]:
from azureml.core import Workspace, Dataset, Datastore

subscription_id = '21ff0fc0-dd2c-450d-93b7-96eeb3699b22'
resource_group = 'project-group-29'
workspace_name = 'project-group-29'

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = Datastore.get(workspace, "workspaceblobstore")
dataset = Dataset.Tabular.from_parquet_files(path=(datastore, 'states/states_sentiment'))
df = dataset.to_spark_dataframe()
df.head()

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 50, 7, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E9E3FSTNM to authenticate.
Interactive authentication successfully completed.
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe', 'activityApp': 'TabularDataset', 'runId': '855e5980-60ba-411f-84e1-006fea505c71'}
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe', 'activityApp': 'TabularDataset', 'runId': '855e5980-60ba-411f-84e1-006fea505c71', 'run_id': '855e5980-60ba-411f-84e1-006fea505c71'}


Row(author='fireweedflowers', text='I live in Colorado these days but I was born and raised in Wyoming and was emphatically a socialist.', controversiality=0, created_utc=1701619261, distinguished=None, edited=None, gilded=0, id='kbtqz1h', parent_id='t3_187jmr5', score=1, subreddit='wyoming', subreddit_id='t5_2r53d', alaska=False, alabama=False, arkansas=False, american samoa=False, arizona=False, california=False, colorado=True, connecticut=False, district of columbia=False, delaware=False, florida=False, georgia=False, guam=False, hawaii=False, iowa=False, idaho=False, illinois=False, indiana=False, kansas=False, kentucky=False, louisiana=False, massachusetts=False, maryland=False, maine=False, michigan=False, minnesota=False, missouri=False, mississippi=False, montana=False, north carolina=False, north dakota=False, nebraska=False, new hampshire=False, new jersey=False, new mexico=False, nevada=False, new york=False, ohio=False, oklahoma=False, oregon=False, pennsylvania=False, puer

In [4]:
#df.printSchema()
df.show()

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 50, 9, Finished, Available, Finished)

+--------------------+--------------------+----------------+-----------+-------------+------+------+-------+----------+-----+-------------------+------------+------+-------+--------+--------------+-------+----------+--------+-----------+--------------------+--------+-------+-------+-----+------+-----+-----+--------+-------+------+--------+---------+-------------+--------+-----+--------+---------+--------+-----------+-------+--------------+------------+--------+-------------+----------+----------+------+--------+-----+--------+------+------------+-----------+------------+--------------+------------+---------+-----+-----+--------+--------------+-------+----------+---------+-------------+-------+-------------+
|              author|                text|controversiality|created_utc|distinguished|edited|gilded|     id| parent_id|score|          subreddit|subreddit_id|alaska|alabama|arkansas|american samoa|arizona|california|colorado|connecticut|district of columbia|delaware|florida|georgia|

In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, VectorAssembler
from pyspark.ml.regression import LinearRegression

state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", 
"Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Guam", 
"Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", 
"Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", 
"Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", 
"New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", 
"Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", 
"Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
state_names = [x.lower() for x in state_names]

state_names.append("sentiment_num")
state_names.append("rawFeatures")

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)
idf = VectorAssembler(inputCols= state_names, outputCol="features")

lr = LinearRegression(featuresCol="features", labelCol="score")

pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lr])

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 50, 17, Finished, Available, Finished)

In [13]:
(trainingData, testData) = df.randomSplit([0.7, 0.3])

model = pipeline.fit(trainingData)
predictions = model.transform(testData)

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 50, 18, Finished, Available, Finished)

In [14]:
datastore = 'azureml://datastores/workspaceblobstore/paths'
model.save(f"{datastore}/models/ml_model_score_prediction")

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 50, 19, Finished, Available, Finished)

In [ ]:
from pyspark.ml import PipelineModel
PipelineModel.load(f"{datastore}/models/ml_model_score_prediction") 

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 50, 20, Submitted, Running, Running)